In [1]:
import pickle
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
train_df = pd.read_csv("../../input/feedback-prize-english-language-learning/train.csv")
train_df.head(2)

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5


In [3]:
model_path_list = [
    #('../../14_Baseline4/exp/result/14_v1_01/oof_df.csv',1), # deberta-v3-base
    ('../../14_Baseline4/exp/result/14_v1_07/oof_df.csv',1), # deberta-v3-base, 10folds    
    #('../../14_Baseline4/exp/result/14_v1_08/oof_df.csv',1), # deberta-base
    #('../../14_Baseline4/exp/result/14_v1_09/oof_df.csv',1), # roberta-base
    #('../../14_Baseline4/exp/result/14_v1_10/oof_df.csv',1), # deberta-v3-large
    #('../../14_Baseline4/exp/result/14_v1_11/oof_df.csv',1), # xlm-roberta-base
    ('../../14_Baseline4/exp/result/14_v1_12/oof_df.csv',1), # deberta-large
    #('../../14_Baseline4/exp/result/14_v1_13/oof_df.csv',1), # roberta-large
    ('../../14_Baseline4/exp/result/14_v1_14/oof_df.csv',1), # xlm-roberta-large
#     ('../../14_Baseline4/exp/result/14_v1_15/oof_df.csv',1), # deberta-v2-xlarge
#     ('../../14_Baseline4/exp/result/14_v1_16/oof_df.csv',1), # deberta-xlarge
    
#     ('../../14_Baseline4/exp/result/14_v2_01/oof_df.csv',1), # deberta-v3-base, seed200
#     ('../../14_Baseline4/exp/result/14_v2_02/oof_df.csv',1), # deberta-v3-large, seed200
#     ('../../14_Baseline4/exp/result/14_v2_03/oof_df.csv',1), # deberta-base, seed200
    
    #('../../17_Pseudo3/exp/result/17_v1_01/oof_df.csv',1), # deberta-v3-base
    #('../../17_Pseudo3/exp/result/17_v1_02/oof_df.csv',1), # deberta-v3-base
    #('../../17_Pseudo3/exp/result/17_v1_03/oof_df.csv',1), # deberta-v3-large
    #('../../17_Pseudo3/exp/result/17_v1_04/oof_df.csv',1), # deberta-v3-base, seed200
    #('../../17_Pseudo3/exp/result/17_v1_05/oof_df.csv',1), # deberta-base
    #('../../17_Pseudo3/exp/result/17_v1_06/oof_df.csv',1), # deberta-large
    #('../../17_Pseudo3/exp/result/17_v1_07/oof_df.csv',1), # deberta-v3-small
    
    #('../../18_Pseudo4/exp/result/18_v1_01/oof_df.csv',1), # deberta-v3-base
    #('../../18_Pseudo4/exp/result/18_v1_02/oof_df.csv',1), # deberta-v3-large
    #('../../18_Pseudo4/exp/result/18_v1_03/oof_df.csv',1), # deberta-v3-small
    
    #('../../19_Distillation/exp/result/19_v1_01/oof_df.csv',1), # deberta-v3-base <- dbv3-l
    ('../../19_Distillation/exp/result/19_v1_02/oof_df.csv',1), # deberta-v3-base <- dbv3-xl
    ('../../19_Distillation/exp/result/19_v1_03/oof_df.csv',1), # deberta-v3-base <- db-xl
    ('../../19_Distillation/exp/result/19_v1_04/oof_df.csv',1), # deberta-v3-small <- dbv3-l
    
    ('../../20_Distillation2/exp/result/20_v2_01/oof_df.csv',1), # deberta-v3-base <- dbv3-b
    #('../../20_Distillation2/exp/result/20_v2_02/oof_df.csv',1), # deberta-v3-base <- dbv3-l
    ('../../20_Distillation2/exp/result/20_v2_03/oof_df.csv',1), # deberta-v3-large <- dbv3-l
]

oof_df_list = [
    pd.read_csv(model_path) for model_path,_ in model_path_list
]
weights_list = [
    w for _,w in model_path_list
]
weights_list = [w/sum(weights_list) for w in weights_list]

for oof_df in oof_df_list:
    oof_df = train_df[['text_id']].merge(oof_df, how='left', on='text_id')
    
num_models = len(model_path_list)

TARGET_COLS = ['cohesion','syntax','vocabulary','phraseology','grammar','conventions']

preds = 0
for oof_df, w in zip(oof_df_list, weights_list):
    preds += oof_df[TARGET_COLS].values * w

oofs = []
for oof_df in oof_df_list:
    oofs.append(oof_df[TARGET_COLS].values)

In [4]:
oof_df = oof_df_list[0].copy()
for i,col in enumerate(TARGET_COLS):
    oof_df[col] = preds[:,i]

In [5]:
import numpy as np

def calc_metric(pred, gt):
    '''
    pred : (num_data, num_labels)
    gt : (num_data, num_labels)
    '''
    score = np.sqrt(np.mean((pred - gt)**2, axis=0))
    score = score.mean()
    return score

In [6]:
score = calc_metric(pred=oof_df[TARGET_COLS].values, gt=train_df[TARGET_COLS].values)
print('CV={:.4f}'.format(score))

CV=0.4440


# Optimize class-wise

In [7]:
ensemble_predictions=np.stack(oofs)
ensemble_predictions.shape

(8, 3911, 6)

In [8]:
from skopt import gp_minimize

def run_optimize(i):
    target_col = TARGET_COLS[i]
    print("*"*50)
    print(target_col)
    print("*"*50)
    def ensemble_score(weights,return_pred=False):
        weights=np.array(weights)
        weights=weights.reshape(-1,1,1)/weights.sum()
        p=weights.reshape(-1,1,1)*ensemble_predictions[:,:,i,None]
        p=p.sum(0)
        score=calc_metric(p, train_df[[target_col]])
        if return_pred:
            return score,p
        else:
            return score   
    results = gp_minimize(ensemble_score, np.array([[0.1,1] for i in range(len(ensemble_predictions))]),
                          verbose=True,random_state=2022)
    best_weights=np.array(results['x'])/sum(results['x'])
    print(best_weights)
    # smaller is better for this metric
    score, ensemble_pred=ensemble_score(best_weights,True)
    print("score={:.4f}", score)
    return score, best_weights

In [9]:
scores = {}
best_weights = {}
for i,target_col in enumerate(TARGET_COLS):
    score, weights = run_optimize(i)
    scores[target_col] = score
    best_weights[target_col] = weights

**************************************************
cohesion
**************************************************
Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.0054
Function value obtained: 0.4725
Current minimum: 0.4725
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.0023
Function value obtained: 0.4723
Current minimum: 0.4723
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 0.0019
Function value obtained: 0.4732
Current minimum: 0.4723
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 0.0018
Function value obtained: 0.4727
Current minimum: 0.4723
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.

Iteration No: 41 ended. Search finished for the next optimal point.
Time taken: 0.4956
Function value obtained: 0.4717
Current minimum: 0.4713
Iteration No: 42 started. Searching for the next optimal point.
Iteration No: 42 ended. Search finished for the next optimal point.
Time taken: 0.4934
Function value obtained: 0.4727
Current minimum: 0.4713
Iteration No: 43 started. Searching for the next optimal point.
Iteration No: 43 ended. Search finished for the next optimal point.
Time taken: 0.4921
Function value obtained: 0.4731
Current minimum: 0.4713
Iteration No: 44 started. Searching for the next optimal point.
Iteration No: 44 ended. Search finished for the next optimal point.
Time taken: 0.5568
Function value obtained: 0.4725
Current minimum: 0.4713
Iteration No: 45 started. Searching for the next optimal point.
Iteration No: 45 ended. Search finished for the next optimal point.
Time taken: 0.5414
Function value obtained: 0.4720
Current minimum: 0.4713
Iteration No: 46 started. Sea

Iteration No: 81 ended. Search finished for the next optimal point.
Time taken: 0.5635
Function value obtained: 0.4712
Current minimum: 0.4712
Iteration No: 82 started. Searching for the next optimal point.
Iteration No: 82 ended. Search finished for the next optimal point.
Time taken: 0.5797
Function value obtained: 0.4736
Current minimum: 0.4712
Iteration No: 83 started. Searching for the next optimal point.
Iteration No: 83 ended. Search finished for the next optimal point.
Time taken: 0.6418
Function value obtained: 0.4712
Current minimum: 0.4712
Iteration No: 84 started. Searching for the next optimal point.
Iteration No: 84 ended. Search finished for the next optimal point.
Time taken: 0.6085
Function value obtained: 0.4732
Current minimum: 0.4712
Iteration No: 85 started. Searching for the next optimal point.
Iteration No: 85 ended. Search finished for the next optimal point.
Time taken: 0.6311
Function value obtained: 0.4712
Current minimum: 0.4712
Iteration No: 86 started. Sea

Iteration No: 21 ended. Search finished for the next optimal point.
Time taken: 0.2319
Function value obtained: 0.4385
Current minimum: 0.4368
Iteration No: 22 started. Searching for the next optimal point.
Iteration No: 22 ended. Search finished for the next optimal point.
Time taken: 0.2723
Function value obtained: 0.4369
Current minimum: 0.4368
Iteration No: 23 started. Searching for the next optimal point.
Iteration No: 23 ended. Search finished for the next optimal point.
Time taken: 0.2641
Function value obtained: 0.4374
Current minimum: 0.4368
Iteration No: 24 started. Searching for the next optimal point.
Iteration No: 24 ended. Search finished for the next optimal point.
Time taken: 0.3173
Function value obtained: 0.4378
Current minimum: 0.4368
Iteration No: 25 started. Searching for the next optimal point.
Iteration No: 25 ended. Search finished for the next optimal point.
Time taken: 0.3387
Function value obtained: 0.4375
Current minimum: 0.4368
Iteration No: 26 started. Sea

Iteration No: 61 ended. Search finished for the next optimal point.
Time taken: 0.4739
Function value obtained: 0.4371
Current minimum: 0.4366
Iteration No: 62 started. Searching for the next optimal point.
Iteration No: 62 ended. Search finished for the next optimal point.
Time taken: 0.4496
Function value obtained: 0.4366
Current minimum: 0.4366
Iteration No: 63 started. Searching for the next optimal point.
Iteration No: 63 ended. Search finished for the next optimal point.
Time taken: 0.4452
Function value obtained: 0.4382
Current minimum: 0.4366
Iteration No: 64 started. Searching for the next optimal point.
Iteration No: 64 ended. Search finished for the next optimal point.
Time taken: 0.4820
Function value obtained: 0.4381
Current minimum: 0.4366
Iteration No: 65 started. Searching for the next optimal point.
Iteration No: 65 ended. Search finished for the next optimal point.
Time taken: 0.4673
Function value obtained: 0.4379
Current minimum: 0.4366
Iteration No: 66 started. Sea

...

Iteration No: 10 ended. Evaluation done at random point.
Time taken: 0.1782
Function value obtained: 0.4645
Current minimum: 0.4634
Iteration No: 11 started. Searching for the next optimal point.
Iteration No: 11 ended. Search finished for the next optimal point.
Time taken: 0.2564
Function value obtained: 0.4642
Current minimum: 0.4634
Iteration No: 12 started. Searching for the next optimal point.
Iteration No: 12 ended. Search finished for the next optimal point.
Time taken: 0.2097
Function value obtained: 0.4635
Current minimum: 0.4634
Iteration No: 13 started. Searching for the next optimal point.
Iteration No: 13 ended. Search finished for the next optimal point.
Time taken: 0.2304
Function value obtained: 0.4632
Current minimum: 0.4632
Iteration No: 14 started. Searching for the next optimal point.
Iteration No: 14 ended. Search finished for the next optimal point.
Time taken: 0.2576
Function value obtained: 0.4637
Current minimum: 0.4632
Iteration No: 15 started. Searching for 

Iteration No: 50 ended. Search finished for the next optimal point.
Time taken: 0.4294
Function value obtained: 0.4639
Current minimum: 0.4631
Iteration No: 51 started. Searching for the next optimal point.
Iteration No: 51 ended. Search finished for the next optimal point.
Time taken: 0.5098
Function value obtained: 0.4643
Current minimum: 0.4631
Iteration No: 52 started. Searching for the next optimal point.
Iteration No: 52 ended. Search finished for the next optimal point.
Time taken: 0.4932
Function value obtained: 0.4649
Current minimum: 0.4631
Iteration No: 53 started. Searching for the next optimal point.
Iteration No: 53 ended. Search finished for the next optimal point.
Time taken: 0.5041
Function value obtained: 0.4644
Current minimum: 0.4631
Iteration No: 54 started. Searching for the next optimal point.
Iteration No: 54 ended. Search finished for the next optimal point.
Time taken: 0.5592
Function value obtained: 0.4631
Current minimum: 0.4631
Iteration No: 55 started. Sea

Iteration No: 90 ended. Search finished for the next optimal point.
Time taken: 0.6392
Function value obtained: 0.4641
Current minimum: 0.4629
Iteration No: 91 started. Searching for the next optimal point.
Iteration No: 91 ended. Search finished for the next optimal point.
Time taken: 0.7045
Function value obtained: 0.4629
Current minimum: 0.4629
Iteration No: 92 started. Searching for the next optimal point.
Iteration No: 92 ended. Search finished for the next optimal point.
Time taken: 0.7926
Function value obtained: 0.4641
Current minimum: 0.4629
Iteration No: 93 started. Searching for the next optimal point.
Iteration No: 93 ended. Search finished for the next optimal point.
Time taken: 0.7575
Function value obtained: 0.4643
Current minimum: 0.4629
Iteration No: 94 started. Searching for the next optimal point.
Iteration No: 94 ended. Search finished for the next optimal point.
Time taken: 0.7399
Function value obtained: 0.4643
Current minimum: 0.4629
Iteration No: 95 started. Sea

Iteration No: 30 ended. Search finished for the next optimal point.
Time taken: 0.3668
Function value obtained: 0.4372
Current minimum: 0.4369
Iteration No: 31 started. Searching for the next optimal point.
Iteration No: 31 ended. Search finished for the next optimal point.
Time taken: 0.3155
Function value obtained: 0.4372
Current minimum: 0.4369
Iteration No: 32 started. Searching for the next optimal point.
Iteration No: 32 ended. Search finished for the next optimal point.
Time taken: 0.3687
Function value obtained: 0.4369
Current minimum: 0.4369
Iteration No: 33 started. Searching for the next optimal point.
Iteration No: 33 ended. Search finished for the next optimal point.
Time taken: 0.4339
Function value obtained: 0.4371
Current minimum: 0.4369
Iteration No: 34 started. Searching for the next optimal point.
Iteration No: 34 ended. Search finished for the next optimal point.
Time taken: 0.3998
Function value obtained: 0.4373
Current minimum: 0.4369
Iteration No: 35 started. Sea

Iteration No: 70 ended. Search finished for the next optimal point.
Time taken: 0.5448
Function value obtained: 0.4368
Current minimum: 0.4367
Iteration No: 71 started. Searching for the next optimal point.
Iteration No: 71 ended. Search finished for the next optimal point.
Time taken: 0.5403
Function value obtained: 0.4369
Current minimum: 0.4367
Iteration No: 72 started. Searching for the next optimal point.
Iteration No: 72 ended. Search finished for the next optimal point.
Time taken: 0.5926
Function value obtained: 0.4375
Current minimum: 0.4367
Iteration No: 73 started. Searching for the next optimal point.
Iteration No: 73 ended. Search finished for the next optimal point.
Time taken: 0.5723
Function value obtained: 0.4368
Current minimum: 0.4367
Iteration No: 74 started. Searching for the next optimal point.
Iteration No: 74 ended. Search finished for the next optimal point.
Time taken: 0.5811
Function value obtained: 0.4374
Current minimum: 0.4367
Iteration No: 75 started. Sea

In [10]:
best_weights

{'cohesion': array([0.28280856, 0.27507774, 0.1193681 , 0.13023989, 0.10766315,
        0.02828086, 0.02828086, 0.02828086]),
 'syntax': array([0.25400963, 0.25400963, 0.16610206, 0.12789301, 0.02540096,
        0.02540096, 0.02540096, 0.12178279]),
 'vocabulary': array([0.26009321, 0.1862044 , 0.12793666, 0.09582997, 0.06913791,
        0.06704498, 0.16774356, 0.02600932]),
 'phraseology': array([0.23354142, 0.1775977 , 0.11885495, 0.13625092, 0.04356667,
        0.02335414, 0.1084238 , 0.1584104 ]),
 'grammar': array([0.21911871, 0.03393248, 0.03393248, 0.13957232, 0.14488871,
        0.03393248, 0.05529807, 0.33932476]),
 'conventions': array([0.20888558, 0.16052861, 0.13095707, 0.12920709, 0.1538307 ,
        0.06980844, 0.02088856, 0.12589396])}

In [11]:
scores

{'cohesion': 0.4711651442144193,
 'syntax': 0.4366087851987312,
 'vocabulary': 0.4059363910249887,
 'phraseology': 0.44598059380469607,
 'grammar': 0.46293298970202357,
 'conventions': 0.4367146419853315}

In [12]:
CV = sum(scores.values()) / len(scores)
print("CV={:.4f}".format(CV))

CV=0.4432


# Error Analysis - Check Corr

In [13]:
cols = ['cohesion','syntax','vocabulary','phraseology','grammar','conventions']
train_df[cols].corr()

,cohesion,syntax,vocabulary,phraseology,grammar,conventions
cohesion,1.000000,0.695459,0.666151,0.690058,0.638689,0.666151
syntax,0.695459,1.000000,0.680562,0.725467,0.709525,0.700025
vocabulary,0.666151,0.680562,1.000000,0.735261,0.654852,0.664292
phraseology,0.690058,0.725467,0.735261,1.000000,0.719746,0.666842
grammar,0.638689,0.709525,0.654852,0.719746,1.000000,0.673301
conventions,0.666151,0.700025,0.664292,0.666842,0.673301,1.000000


In [14]:
cols = ['cohesion','syntax','vocabulary','phraseology','grammar','conventions']
oof_df[cols].corr()

,cohesion,syntax,vocabulary,phraseology,grammar,conventions
cohesion,1.000000,0.955118,0.926265,0.921866,0.837394,0.897350
syntax,0.955118,1.000000,0.947758,0.967917,0.927598,0.935000
vocabulary,0.926265,0.947758,1.000000,0.966624,0.887521,0.903522
phraseology,0.921866,0.967917,0.966624,1.000000,0.949834,0.893938
grammar,0.837394,0.927598,0.887521,0.949834,1.000000,0.867061
conventions,0.897350,0.935000,0.903522,0.893938,0.867061,1.000000
